In [11]:
import tensorflow as tf
import numpy as np
a = tf.constant([2,3], dtype= tf.int32)
a = tf.range(1,10,delta=2)
a = tf.linspace(0,10,20)
a = tf.eye(3,3)
a = tf.linalg.diag(np.arange(12).tolist())

In [9]:
#tf.slice(input,begin_vector,size_vector)
tf.slice(tf.reshape(tf.range(12),[3,4]),[1,0],[2,2]).numpy()

array([[4, 5],
       [8, 9]])

In [28]:
# for assign operator
# first we need to set a variable
a = tf.Variable(np.zeros([4,4]), dtype= tf.float16)

In [30]:
a[0:2].assign(tf.ones([2,4],dtype=tf.float16)) 

<tf.Variable 'UnreadVariable' shape=(4, 4) dtype=float16, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float16)>

In [ ]:
#省略号可以表示多个冒号
tf.print(a[...,1])

考虑班级成绩册的例子，有4个班级，每个班级10个学生，每个学生7门科目成绩。可以用一个4×10×7的张量来表示。

In [31]:
scores = tf.random.uniform((4,10,7),minval=0,maxval=100,dtype=tf.int32)

In [ ]:
#抽取每个班级第0个学生，第5个学生，第9个学生的全部成绩
p = tf.gather(scores,[0,5,9],axis=1)
tf.print(p)

In [ ]:
#抽取每个班级第0个学生，第5个学生，第9个学生的第1门课程，第3门课程，第6门课程成绩
q = tf.gather(tf.gather(scores,[0,5,9],axis=1),[1,3,6],axis=2)
tf.print(q)

In [33]:
# 抽取第0个班级第0个学生，第2个班级的第4个学生，第3个班级的第6个学生的全部成绩
#indices的长度为采样样本的个数，每个元素为采样位置的坐标
tf.gather_nd(scores,indices = [(0,0),(2,4),(3,6)])

<tf.Tensor: shape=(3, 7), dtype=int32, numpy=
array([[74, 37, 10, 92, 54, 58,  0],
       [37, 68, 52, 55, 45, 42, 60],
       [11, 11, 62, 97, 12, 76, 83]])>

In [35]:
tf.gather_nd(scores,indices = [(0,0,0),(2,4,1),(3,6,2)])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([74, 68, 62])>

In [ ]:
#抽取每个班级第0个学生，第5个学生，第9个学生的全部成绩
p = tf.boolean_mask(scores,[True,False,False,False,False,
                            True,False,False,False,True],axis=1)
tf.print(p)

In [ ]:
#抽取第0个班级第0个学生，第2个班级的第4个学生，第3个班级的第6个学生的全部成绩
s = tf.boolean_mask(scores,
    [[True,False,False,False,False,False,False,False,False,False],
     [False,False,False,False,False,False,False,False,False,False],
     [False,False,False,False,True,False,False,False,False,False],
     [False,False,False,False,False,False,True,False,False,False]])
tf.print(s)

In [ ]:
tf.print(tf.boolean_mask(c,c<0),"\n") 
tf.print(c[c<0]) #布尔索引，为boolean_mask的语法糖形式

以上这些方法仅能提取张量的部分元素值，但不能更改张量的部分元素值得到新的张量。

如果要通过修改张量的部分元素值得到新的张量，可以使用tf.where和tf.scatter_nd。

tf.where可以理解为if的张量版本，此外它还可以用于找到满足条件的所有元素的位置坐标。

tf.scatter_nd的作用和tf.gather_nd有些相反，tf.gather_nd用于收集张量的给定位置的元素，

而tf.scatter_nd可以将某些值插入到一个给定shape的全0的张量的指定位置处

In [36]:
#找到张量中小于0的元素,将其换成np.nan得到新的张量
#tf.where和np.where作用类似，可以理解为if的张量版本

c = tf.constant([[-1,1,-1],[2,2,-2],[3,-3,3]],dtype=tf.float32)
d = tf.where(c<0,tf.fill(c.shape,np.nan),c) 
d

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[nan,  1., nan],
       [ 2.,  2., nan],
       [ 3., nan,  3.]], dtype=float32)>

In [37]:
#如果where只有一个参数，将返回所有满足条件的位置坐标
indices = tf.where(c<0)
indices

<tf.Tensor: shape=(4, 2), dtype=int64, numpy=
array([[0, 0],
       [0, 2],
       [1, 2],
       [2, 1]], dtype=int64)>

In [39]:
c = tf.constant([[-1,1,-1],[2,2,-2],[3,-3,3]],dtype=tf.float32)
d = tf.where(c<0,tf.fill(c.shape,np.nan),tf.fill(c.shape,1.0)) 
d

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[nan,  1., nan],
       [ 1.,  1., nan],
       [ 1., nan,  1.]], dtype=float32)>

In [40]:
#将张量的第[0,0]和[2,1]两个位置元素替换为0得到新的张量
d = c - tf.scatter_nd([[0,0],[2,1]],[c[0,0],c[2,1]],c.shape)
d

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.,  1., -1.],
       [ 2.,  2., -2.],
       [ 3.,  0.,  3.]], dtype=float32)>

In [41]:
tf.scatter_nd([[0,0],[2,1]],[c[0,0],c[2,1]],c.shape)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-1.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0., -3.,  0.]], dtype=float32)>

In [43]:
tf.scatter_nd([[0,0],[2,1]],[5,5],c.shape)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[5, 0, 0],
       [0, 0, 0],
       [0, 5, 0]])>

In [ ]:
scatter_nd的作用和gather_nd有些相反
#可以将某些值插入到一个给定shape的全0的张量的指定位置处。
indices = tf.where(c<0)
tf.scatter_nd(indices,tf.gather_nd(c,indices),c.shape)

In [ ]:
tf.reshape 可以改变张量的形状。

tf.squeeze 可以减少维度。

tf.expand_dims 可以增加维度。

tf.transpose 可以交换维度。

In [ ]:
tf.concat和tf.stack有略微的区别，tf.concat是连接，不会增加维度，而tf.stack是堆叠，会增加维度。

In [ ]:
tf.split是tf.concat的逆运算，可以指定分割份数平均分割，也可以通过指定每份的记录数量进行分割。

In [44]:
#tf.split(value,num_or_size_splits,axis)
tf.split(c,3,axis = 0)  #指定分割份数，平均分割

[<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-1.,  1., -1.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 2.,  2., -2.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 3., -3.,  3.]], dtype=float32)>]

In [48]:
c = tf.random.uniform([12,12])
tf.split(c,[1,1,-1],axis = 0) #指定每份的记录数量

[<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
 array([[0.8935895 , 0.895692  , 0.91402435, 0.69897413, 0.7914586 ,
         0.3803165 , 0.2065692 , 0.21990776, 0.5720501 , 0.3484335 ,
         0.3429693 , 0.68787324]], dtype=float32)>,
 <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
 array([[0.6472976 , 0.82212925, 0.11641526, 0.5760038 , 0.7230698 ,
         0.9944571 , 0.31512165, 0.85974634, 0.4827547 , 0.67862284,
         0.92354727, 0.14661741]], dtype=float32)>,
 <tf.Tensor: shape=(10, 12), dtype=float32, numpy=
 array([[0.99819493, 0.5212045 , 0.88263226, 0.8542752 , 0.57050717,
         0.42869663, 0.03776813, 0.7298223 , 0.8275517 , 0.70772684,
         0.5436336 , 0.26363873],
        [0.53545046, 0.7393999 , 0.18041193, 0.3841871 , 0.30740607,
         0.28331494, 0.2041887 , 0.6020137 , 0.8725177 , 0.28952563,
         0.00612175, 0.7088337 ],
        [0.13641608, 0.81328964, 0.39551198, 0.66780996, 0.80712867,
         0.6830174 , 0.40002465, 0.9775413 , 0.28007185,

In [49]:
x = tf.constant([0.9,-0.8,100.0,-20.0,0.7])
y = tf.clip_by_value(x,clip_value_min=-1,clip_value_max=1)
z = tf.clip_by_norm(x,clip_norm = 3)
tf.print(y)
tf.print(z)

[0.9 -0.8 1 -1 0.7]
[0.0264732055 -0.0235317405 2.94146752 -0.588293493 0.0205902718]


In [54]:
x/tf.norm(x)*3

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.02647321, -0.02353174,  2.9414675 , -0.5882935 ,  0.02059027],
      dtype=float32)>

In [ ]:
向量运算符只在一个特定轴上运算，将一个向量映射到一个标量或者另外一个向量。 许多向量运算符都以reduce开头。

In [ ]:
#向量reduce
a = tf.range(1,10)
tf.print(tf.reduce_sum(a))
tf.print(tf.reduce_mean(a))
tf.print(tf.reduce_max(a))
tf.print(tf.reduce_min(a))
tf.print(tf.reduce_prod(a))

In [58]:
a = tf.reshape(tf.range(1,5),[2,2])

In [59]:
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>

In [61]:
tf.print(tf.reduce_prod(a,0))

[3 8]


In [ ]:
#张量指定维度进行reduce
b = tf.reshape(a,(3,3))
tf.print(tf.reduce_sum(b, axis=1, keepdims=True))
tf.print(tf.reduce_sum(b, axis=0, keepdims=True))

In [ ]:
#bool类型的reduce
p = tf.constant([True,False,False])
q = tf.constant([False,False,True])
tf.print(tf.reduce_all(p))
tf.print(tf.reduce_any(q))

In [64]:
c = tf.reshape(tf.range(24),[6,4])
#利用tf.foldr实现tf.reduce_sum
s = tf.foldr(lambda a,b:a+b,c) 
tf.print(s)

[60 66 72 78]


In [65]:
c = tf.reshape(tf.range(24),[6,4])
#利用tf.foldr实现tf.reduce_sum
s = tf.foldl(lambda a,b:a+b,c) 
tf.print(s)

[60 66 72 78]


In [ ]:
至于tf.foldr()和这个函数是基本上一样的，无非就是从右边开始计算到左边而已。

In [ ]:
#cum扫描累积
a = tf.range(1,10)
tf.print(tf.math.cumsum(a))
tf.print(tf.math.cumprod(a))

In [67]:
#tf.math.top_k可以用于对张量排序
a = tf.constant([1,3,7,5,4,8])

values,indices = tf.math.top_k(a,3,sorted=False)
tf.print(values)
tf.print(indices)

[5 8 7]
[3 5 2]


In [ ]:
#矩阵乘法
a = tf.constant([[1,2],[3,4]])
b = tf.constant([[2,0],[0,2]])
a@b  #等价于tf.matmul(a,b)

In [ ]:
tf.transpose/ tf.linalg.inv/ tf.linalg.trace/ tf.linalg.eigvals/ tf.linalg.det
tf.linalg.svd

In [ ]:
#broadcast
a = tf.constant([1,2,3])
b = tf.constant([[0,0,0],[1,1,1],[2,2,2]])
b + a  # Identical to b + tf.broadcast_to(a,b.shape)
tf.broadcast_to(a,b.shape)

In [ ]:
一，Autograph编码规范总结
1，被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

2，避免在@tf.function修饰的函数内部定义tf.Variable.

3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

In [77]:
import tensorflow as tf
import numpy as np 
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a+b
    print("tracing")
    return c

In [78]:
myadd(tf.constant("helo"), tf.constant("ak"))

0
1
2
tracing


<tf.Tensor: shape=(), dtype=string, numpy=b'heloak'>

In [79]:
import tensorflow as tf
import numpy as np 
@tf.function
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a+b
    print("tracing")
    return c

In [80]:
myadd(tf.constant("helo"), tf.constant("ak"))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'heloak'>

In [81]:
myadd(tf.constant("helo"), tf.constant("ak"))

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'heloak'>

发生了2件事情，

第一件事情是创建计算图。

即创建一个静态计算图，跟踪执行一遍函数体中的Python代码，确定各个变量的Tensor类型，并根据执行顺序将算子添加到计算图中。 在这个过程中，如果开启了autograph=True(默认开启),会将Python控制流转换成TensorFlow图内控制流。 主要是将if语句转换成 tf.cond算子表达，将while和for循环语句转换成tf.while_loop算子表达，并在必要的时候添加 tf.control_dependencies指定执行顺序依赖关系。

第二件事情是执行计算图


当我们再次用相同的输入参数类型调用这个被@tf.function装饰的函数时，后面到底发生了什么？
只会发生一件事情，那就是上面步骤的第二步，执行计算图。

由于输入参数的类型已经发生变化，已经创建的计算图不能够再次使用。

需要重新做2件事情：创建新的计算图、执行计算图。

Note: if the data type of the argument is not Tensor in the original definition of this function, then the graph will be re-created each time after calling this function.

TensorFlow提供了一个基类tf.Module，通过继承它构建子类，我们不仅可以获得以上的自然而然，而且可以非常方便地管理变量，还可以非常方便地管理它引用的其它Module，最重要的是，我们能够利用tf.saved_model保存模型并实现跨平台部署使用。

实际上，tf.keras.models.Model,tf.keras.layers.Layer 都是继承自tf.Module的，提供了方便的变量管理和所引用的子模块管理的功能。

因此，利用tf.Module提供的封装，再结合TensoFlow丰富的低阶API，实际上我们能够基于TensorFlow开发任意机器学习模型(而非仅仅是神经网络模型)，并实现跨平台部署使用

# 应用tf.Module封装Autograph

In [ ]:
class DemoModule(tf.Module):
    def __init__(self,init_value = tf.constant(0.0),name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:  #相当于with tf.name_scope("demo_module")
            self.x = tf.Variable(init_value,dtype = tf.float32,trainable=True)

     
    @tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])  
    def addprint(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return(self.x)
#执行
demo = DemoModule(init_value = tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))